# Import librairies et modules

In [1]:

import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
dbname = os.getenv("DB_NAME")

DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{dbname}?sslmode=require"

def fetch_table_from_trafic_db(table_name) :
    '''
        Récupère toutes les données d'une table spécifique depuis la base de données de trafic.
        
        Args:
            table_name (str): Le nom de la table à récupérer.
            
        Returns:
            pd.DataFrame: Un DataFrame contenant les données de la table demandée.
    '''
    engine = create_engine(DATABASE_URL)
    query = f"SELECT * FROM {table_name};" 
    df = pd.read_sql(query, engine)
    return df


# Import df

In [ ]:
df_events = fetch_table_from_trafic_db('events_data')
df_matching_troncon = fetch_table_from_trafic_db('matching_event_troncon')
df_pollution_data = fetch_table_from_trafic_db('pollution_data')
df_trafic = fetch_table_from_trafic_db('trafic_routier')
df_weather_data = fetch_table_from_trafic_db('weather_data')
df_holidays = fetch_table_from_trafic_db('weekday_holidays')

# Jointures

## Trafic + holidays

### Explo df_holidays

In [5]:
df_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   dates        685 non-null    object
 1   is_vacances  685 non-null    int64 
 2   is_ferie     685 non-null    int64 
 3   is_samedi    685 non-null    int64 
 4   is_dimanche  685 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 26.9+ KB


In [6]:
df_holidays.head(2)

,dates,is_vacances,is_ferie,is_samedi,is_dimanche
0,2025-02-15,1,0,1,0
1,2025-02-16,1,0,0,1


In [7]:
df_holidays['dates'] = pd.to_datetime(df_holidays['dates'])

In [9]:
df_holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 685 entries, 0 to 684
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   dates        685 non-null    datetime64[ns]
 1   is_vacances  685 non-null    int64         
 2   is_ferie     685 non-null    int64         
 3   is_samedi    685 non-null    int64         
 4   is_dimanche  685 non-null    int64         
dtypes: datetime64[ns](1), int64(4)
memory usage: 26.9 KB


### Explo df_trafic

In [10]:
df_trafic.head(2)

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,geo_point_2d,geometrie,shape_geo,horodatage,type_geo,coordinates_geo
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,"{47.23414730688424,-1.5808786419612229}","{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}"
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,"{47.23083661767159,-1.5704468983375661}","{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}"


In [11]:
df_trafic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1554761 entries, 0 to 1554760
Data columns (total 16 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   id_technique       1554761 non-null  object        
 1   id                 1554761 non-null  int64         
 2   debit              1554761 non-null  int64         
 3   longueur           1554761 non-null  int64         
 4   taux_occupation    1554761 non-null  float64       
 5   code_couleur       1554761 non-null  int64         
 6   nom_du_troncon     1554761 non-null  object        
 7   etat_du_trafic     1554761 non-null  object        
 8   temps_de_parcours  1554761 non-null  int64         
 9   vitesse            1554761 non-null  int64         
 10  geo_point_2d       1554761 non-null  object        
 11  geometrie          1554761 non-null  object        
 12  shape_geo          1554761 non-null  object        
 13  horodatage         1554761 

In [14]:
df_trafic_holidays = pd.merge(df_trafic, df_holidays,
                              how = 'left',
                              left_on = df_trafic['horodatage'].dt.date,
                              right_on = 'dates')

ValueError: You are trying to merge on object and datetime64[ns] columns for key 'dates'. If you wish to proceed you should use pd.concat

In [13]:
df_trafic_holidays

,id_technique,id,debit,longueur,taux_occupation,code_couleur,nom_du_troncon,etat_du_trafic,temps_de_parcours,vitesse,...,geometrie,shape_geo,horodatage,type_geo,coordinates_geo,dates,is_vacances,is_ferie,is_samedi,is_dimanche
0,772-20250218T205200,772,120,410,1.2,3,Vannes I9,Fluide,62,24,...,"{{-1.583123434893785,47.235185973808406},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5808786419612229,47.23414730688424}",NaT,NaN,NaN,NaN,NaN
1,158-20250218T205200,158,300,364,3.7,3,Le Lasseur P2,Fluide,63,21,...,"{{-1.568589501648445,47.23188191655151},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.5704468983375661,47.23083661767159}",NaT,NaN,NaN,NaN,NaN
2,497-20250218T205200,497,180,224,2.0,3,Lauriol I4,Fluide,37,22,...,"{{-1.563539784805616,47.236320687488636},{-1.5...",LineString,2025-02-18 20:52:00,Point,"{-1.5635261418937887,47.23531301184315}",NaT,NaN,NaN,NaN,NaN
3,5444-20250218T205200,5444,240,177,2.9,3,Pasteur P1,Fluide,40,16,...,"{{-1.581033846171769,47.21286509290865},{-1.57...",LineString,2025-02-18 20:52:00,Point,"{-1.579953397082114,47.21255561611825}",NaT,NaN,NaN,NaN,NaN
4,5053-20250218T205200,5053,60,224,0.8,3,Barbusse P2,Fluide,44,18,...,"{{-1.551553912867712,47.22760386550291},{-1.55...",LineString,2025-02-18 20:52:00,Point,"{-1.5524880999813486,47.22705355840811}",NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1554756,100079-2025-02-16 20:22:00.000,100079,-1,170,-1.0,2,HALIMI I3,Indéterminé,-1,-1,...,"{{-1.556540417989486,47.202596132036895},{-1.5...",LineString,2025-02-16 20:22:00,Point,"{-1.5558260012646832,47.201970536933906}",NaT,NaN,NaN,NaN,NaN
1554757,100085-2025-02-16 20:22:00.000,100085,-1,170,-1.0,2,HALIMI P1,Indéterminé,-1,-1,...,"{{-1.555900533371478,47.203148861695766},{-1.5...",LineString,2025-02-16 20:22:00,Point,"{-1.555339623234197,47.2024468700323}",NaT,NaN,NaN,NaN,NaN
1554758,10074-2025-02-16 20:22:00.000,10074,120,877,1.6,3,VM 723,Fluide,156,20,...,"{{-1.453835196195046,47.28261934850643},{-1.45...",LineString,2025-02-16 20:22:00,Point,"{-1.4491568739688754,47.28495686849942}",NaT,NaN,NaN,NaN,NaN
1554759,100086-2025-02-16 20:22:00.000,100086,120,100,1.6,3,CA FOSSE VERS OUEST,Fluide,20,18,...,"{{-1.563074348836539,47.21034378952278},{-1.56...",LineString,2025-02-16 20:22:00,Point,"{-1.5623101070887704,47.21063748300016}",NaT,NaN,NaN,NaN,NaN
